In [141]:
import os
import sys
import json

import cv2
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

sys.path.append('..')
import data
import model
import utils

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [142]:
params = utils.yaml_to_dict('../config.yml')
params['data_dir'] = os.path.join('..', params['data_dir'])
params['model_dir'] = os.path.join('..', params['model_dir'])
params['videos_folder'] = os.path.join('..', params['videos_folder'])
params['feature_map_folder'] = os.path.join('..', params['feature_map_folder'])
params['json_data_path'] = os.path.join('..', params['json_data_path'])
params['json_metadata_path'] = os.path.join('..', params['json_metadata_path'])

In [143]:
with open(params['json_metadata_path']) as data_file:
    metadata_json = json.load(data_file)

label_by_idx = {
    'level_3': {},
    'level_2': {},
    'level_1': {},
    'level_0': {}
}

In [144]:
for key, item in metadata_json.items():
    if key != 'classes_amount':
        label_by_idx['level_3'][item['idx']] = key
        for level in ['level_2', 'level_1', 'level_0']:
            label_by_idx[level][item[level]['idx']] = item[level]['name']


In [149]:
data_gen_test = data.DataWindowGenerator(params, 'validation')

In [150]:
estimator = tf.estimator.Estimator(
    # Custom model function
    model_fn=model.model_fn,
    params=params,
    # Model directory
    model_dir=params['model_dir'],
    # warm_start_from=cfg.PRE_TRAIN,
    config=tf.estimator.RunConfig(
        keep_checkpoint_max=params['keep_checkpoint_max'],
        save_checkpoints_steps=params['save_checkpoints_steps'],
        save_summary_steps=params['save_summary_steps'],
        log_step_count_steps=params['log_step_count_steps']
    )
)

In [151]:
predictions = estimator.predict(
    input_fn = lambda: data.input_fn(data_gen_test, False, params)
)

In [530]:
with open('../data/predicted_output.json', 'w') as outfile:
    json.dump(prediction_format, outfile)
    

In [529]:
len(prediction_format['results'])

176